#### pb保存的方法

使用tf.train.Saver会保存运行TensorFlow程序所需要的全部信息，然而有时候并不需要某些信息。比如在测试或者离线预测时，只需要知道如何从神经网络的输入层经过前向传播计算得到输出层即可，而不是要类似于变量初始化、模型保存等辅助节点信息。而且，将变量取值和计算图结构分为不同的文件存储有时候不方便，于是TensorFlow提供了convert_variable_to_constant函数，通过这个函数可以将计算图中的变量及其取值通过常量的方式保存，这样整个TensorFlow计算图可以统一放在一个文件中。

In [7]:
import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.platform import gfile

In [6]:
tf.reset_default_graph()

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="v2")
result = v1 + v2

with tf.Session() as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    # 导出当前计算图的GraphDef部分，只需要这一部分就可以完成从输入层到输出层的计算过程
    graph_def = tf.get_default_graph().as_graph_def()
    
    # 将计算图中的变量及其取值转化为常量，同时将图中不必要的节点去掉
    # 一些系统运算也会被转化为计算图中的节点(比如变量初始化操作)
    # 如果只关心程序中定义的某些计算时，和这些计算无关的节点就没有必要导出并保存了。
    # 在下面一行代码中，最后一个参数['add']给出了需要保存的节点名称。add节点是上面
    # 定义的两个变量相加的操作。注意这里给出的是计算节点的名称，所以没有后面的:0；
    # 张量的名称后面有:0, 表示是某个计算节点的第一个输出；而计算节点本身的名称后是
    # 没有:0的
    # convert_variables_to_constants()函数意义：如果您有一个包含变量(Variable)
    # 操作的计算图，那么可以将它们全部转化成相同的const操作。并返回原计算图Graph_def
    # 的简化版本。
    output_graph_def = graph_util.convert_variables_to_constants(
        sess, graph_def, ['add'])
    
    # 将导出的模型存入文件中
    with tf.gfile.GFile("Saved_model/combined_model.pb", "wb") as f:
        f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Froze 2 variables.
Converted 2 variables to const ops.


#### 加载pd文件

通过下面的程序可以直接计算定义的加法运算的结果。当只需要得到计算图中某个节点的取值时，这提供了一个更加方便的方法

tf.import_graph_def的含义：
原型：
```
import_graph_def(
    graph_def, # 一个GraphDef原型
    input_map=None, # 需要返回的张量的名称列表
    return_elements=None,
    name=None,
    op_dict=None,
    producer_op_list=None
)
```
将图从graph_def导入当前默认的Graph中.此函数提供了一种导入"序列化的TensorFlow GraphDef 协议缓冲区"的方法，并将GraphDef中的各个参数提取为tf.Tensor和tf.Operation对象。一旦提取，这些对象将被放置到当前的默认Graph中。参考tf.Graph.as_graph_def创建一个GraphDef模型。
该函数返回来自导入图的操作或张量对象列表，对应return_elements中的名称

In [8]:
tf.reset_default_graph()

with tf.Session() as sess:
    model_filename = "Saved_model/combined_model.pb"
    
    # 读取保存的模型，并将文件解析成对应的GraphDef Protocol Buffer
    with gfile.FastGFile(model_filename, "rb") as f:
        graph_def = tf.GraphDef() # 这个函数表示什么意思
        graph_def.ParseFromString(f.read())
        
    # 将graph_def中保存的图加载到当前的图中。
    # return_elements=["add:0"]给出了返回的张量的名称。
    # 在保存的时候给出的是计算图的名称，所以为"add"。
    # 在加载的时候给出的时张量的名称，所以是add:0
    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    print(sess.run(result))

[array([ 3.], dtype=float32)]
